In [43]:
import torch, torchtune, yaml
from torchtune import config, utils
from generate import InferenceRecipe
from omegaconf import DictConfig
from matplotlib import pyplot as plt

In [2]:
adapter = torch.load('checkpoints/Llama-3-LLaMS/adapter_0.pt')

In [3]:
weights = torch.load('checkpoints/Llama-3-LLaMS/meta_model_4.pt')

In [4]:
with open('./configs/llams_base_generation.yaml') as f:
    config_dict = DictConfig(yaml.safe_load(f))
config_dict

{'model': {'_component_': 'torchtune.models.llama3.llama3_8b'}, 'checkpointer': {'_component_': 'torchtune.utils.FullModelMetaCheckpointer', 'checkpoint_dir': 'checkpoints/Llama-3-LLaMS', 'checkpoint_files': ['meta_model_4.pt'], 'output_dir': 'checkpoints/Llama-3-LlaMS', 'model_type': 'LLAMA3'}, 'device': 'cuda', 'dtype': 'bf16', 'seed': 1234, 'tokenizer': {'_component_': 'torchtune.models.llama3.llama3_tokenizer', 'path': 'checkpoints/Meta-Llama-3-8B/original/tokenizer.model'}, 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: 세종대왕은 대신들 때문에 화가 나서 맥북을 던졌다. ### Response:', 'max_new_tokens': 300, 'temperature': 0.6, 'top_k': 300, 'quantizer': None}

In [5]:
a = InferenceRecipe(config_dict)

DEBUG:torchtune.utils.logging:Setting manual seed to local seed 1234. Local seed is seed + rank = 1234 + 0


In [6]:
a.setup(config_dict)

INFO:torchtune.utils.logging:Model is initialized with precision torch.bfloat16.


In [7]:
tokens = a._tokenizer.encode(config_dict.prompt, add_bos=True, add_eos=False)

In [8]:
prompt = torch.tensor(tokens, dtype=torch.int, device=a._device)

In [19]:
logits = a._model(prompt.view(1, -1), torch.arange(0, 45, device='cuda'))

In [35]:
logits = logits[0, -1]

IndexError: too many indices for tensor of dimension 1

In [37]:
v, t = torch.topk(logits, 10)

In [38]:
pivot = v.select(-1, -1).unsqueeze(-1)

In [39]:
logits = torch.where(logits < pivot, -float("inf"), logits)

In [41]:
probs = torch.nn.functional.softmax(logits, dim=-1)

In [55]:
q = torch.empty_like(probs).exponential_(1)

In [56]:
sample = torch.argmax(probs / q, dim=-1, keepdim=True).to(dtype=torch.int)

In [65]:
a._tokenizer.decode(torch.cat([prompt, sample]).tolist())

'Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: 세종대왕은 대신들 때문에 화가 나서 맥북을 던졌다. ### Response: 이는'

In [51]:
input_pos = torch.tensor([prompt.size(0)])

In [53]:
prompt.view(1, -1)

tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
             13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
             13,  17010,  30151,     25, 101852, 102757,  67945, 108668,  34804,
          62060,  83628,  65950, 109644, 104323,  20565,  74618,  27796, 124191,
         103825,  18359, 101012,    246, 111340,     13,  17010,   6075,     25]],
       device='cuda:0', dtype=torch.int32)